# Voice Assistant

In [ ]:
# Importing necessary libraries and modules
from gtts import gTTS    # for text-to-speech conversion
import os
import speech_recognition as sr   # for speech recognition
import pygame    # for playing audio
import time
import datetime
import pywhatkit  # for playing music on YouTube
import webbrowser
import requests
from bs4 import BeautifulSoup # for web scraping
import random
from dateutil import parser
import smtplib    # for sending emails
import json


# Function to convert text to speech and play the audio
def speak(text):
    
    # Convert text to speech and save it as an MP3 file
    tts = gTTS(text)
    tts.save("output.mp3")
    
    # Initialize and play the audio using pygame
    pygame.mixer.init()
    pygame.mixer.music.load("output.mp3")
    pygame.mixer.music.play()
    
    # Wait for the audio to finish playing
    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)

        
# Function to listen to the user's voice input        
def listen():
    
    recognizer = sr.Recognizer()
    
    with sr.Microphone() as source:
        try:
            recognizer.adjust_for_ambient_noise(source)
            print("Listening....")
            audio = recognizer.listen(source)
            command = recognizer.recognize_google(audio)
            return command
        
        except sr.UnknownValueError:
            print("Sorry, I didn't understand that")
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")
        return ""
    

# Function to greet
def greet():
    
    responses = ["Hi there! How can I help you?", "Hey! What's on your mind?"]
    speak(random.choice(responses))

# Function to respond to well-being inquiries
def respond_to_well_being():
    
    responses = ["I'm just a computer program, but thanks for asking!",
                "I'm doing well, Thank you."]
    speak(random.choice(responses))
       
# Function to get the current time and speak it    
def get_time():
    
    current_time = datetime.datetime.now().strftime("%H :%M")
    speak(f" The time is {current_time}")
    
    
# Function to play music based on user's voice command
def play_music():
    
    speak("Sure, What music would you like to listen?")
    command = listen()
    
    if command :
        pywhatkit.playonyt(command)
        speak(f"Here are the search results for {command} on YouTube. Enjoy your music!")
    else:
        speak("Sorry! I didn't catch it. Please try again after some time.")
     
    
# Function to perform a web search based on user's voice command    
def search():
    
    speak("What would you like to search for?")
    command = listen()

    if command:
        encoded_command = "+".join(command.split())
        webbrowser.open(f"https://www.google.com/search?q={encoded_command}")

        speak(f"Here are the search results for {command}.")
      
    else:
        speak("Sorry! I didn't catch that. Please try again after sometime.")

        
# Function to fetch and speak a random riddle from a website        
def get_random_riddles():
    
    url = "https://www.splashlearn.com/blog/50-best-riddles-for-kids-of-all-grades-with-answers/"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Assuming each riddle is inside a list item within an ordered list
        riddle_items = soup.select("ol li")
        
        # Extracting text from each list item
        riddles = [riddle_item.text.strip() for riddle_item in riddle_items]
        
        random.shuffle(riddles)
        
        # Select the first (now random) riddle
        random_riddle = riddles[0]
        
        return [random_riddle]
        
    else:
        return "Sorry, an unexpected error occurred while fetching riddles."

    
# Function to set a reminder based on user's input    
def set_reminder():

    speak("Sure, what should I remind you about?")
    reminder_text = listen()
    print(f"Reminder: {reminder_text}")
    
    speak("When should I remind you?")
    time_input = listen()
    print(f"Time: {time_input}")

    if reminder_text and time_input:
        
        try:
                          
            # Parse the input time and calculate the time difference
            reminder_time = parser.parse(time_input)
            time_delta = (reminder_time - datetime.datetime.now()).total_seconds()
            
            if time_delta > 0:
                
                # Schedule the reminder and wait for the specified time
                speak(f"I will remind you about '{reminder_text}' at {reminder_time.strftime('%H:%M %p')}.")
                time.sleep(time_delta)
                speak(f"Reminder: {reminder_text}")
                
            else:
                
                speak("Sorry, the specified time is in the past. Please provide a future time.")

        except ValueError:
            
            speak("Sorry, I couldn't understand the time. Please try again.")
            
# Function to send email            
def send_email(to_email, subject, body):
    
    sender_email = "selvabha.56@gmail.com"
    app_password = "pixm buju dbrw bgrt"
    
    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()
            server.login(sender_email, app_password)

            message = f"Subject: {subject}\n\n{body}"
            server.sendmail(sender_email, to_email, message)
            
        print("Email sent successfully!")
    
    except Exception as e:
        
        print(f"Error sending email:{e}")

to_email = "bharathiselvam05@gmail.com"
subject = "Test Subject"
body = "This is a test email sent from the voice assistant."

   
# Function to get weather report 
def get_weather(city):
    
    api_key = "5b050d9e770a90d2cbc6960fba8845e5"

    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": api_key,
        "units": "metric"  
    }

    try:
        response = requests.get(base_url, params = params)
        data = response.json()
        
        print(data)

        if data["cod"] == "404":
            
            speak("City not found. Please check the city name and try again.")
            
        else:
            
            temperature = data["main"]["temp"]
            description = data["weather"][0]["description"]
            speak(f"The current temperature in {city} is {temperature} degrees Celsius, with {description}.")
            
    except Exception as e:
        speak(f"Error fetching weather information: {e}")
        
# Function to get and speak a general knowledge question
def get_and_speak_general_knowledge_question():
    
    api_url = "https://opentdb.com/api.php?amount=1&type=multiple"
    response = requests.get(api_url)

    if response.status_code == 200:
        data = json.loads(response.text)
        question = data["results"][0]["question"]
        options = data["results"][0]["incorrect_answers"] + [data["results"][0]["correct_answer"]]
        
        user_answer_index = None

        # Speak the question
        speak(question)
        print(question)
        
        # Pause to allow the user to listen before speaking the options
        time.sleep(2)
        
        # Speak the options
        for idx, option in enumerate(options, 1):
            speak(f"Option {idx}: {option}")
            print(f"Option {idx}: {option}")
            time.sleep(1)  # Pause between options

        # Get user response
        user_response = listen()
        print(user_response)

        correct_answer_index = options.index(options[-1]) + 1  # Correct answer index

        if 'option 1' in user_response or 'option one' in user_response:
            user_answer_index = 1
        elif 'option 2' in user_response or 'option two' in user_response:
            user_answer_index = 2
        elif 'option 3' in user_response or 'option three' in user_response:
            user_answer_index = 3
        elif 'option 4' in user_response or 'option four' in user_response:
            user_answer_index = 4
        else:
            user_answer_index = None

        if user_answer_index is not None:
            if user_answer_index == correct_answer_index:
                speak("Congratulations! That's the correct answer.")
                print(f"Option {correct_answer_index}: {options[correct_answer_index - 1]}")
            else:
                speak(f"Sorry, the correct answer is Option {correct_answer_index}: {options[correct_answer_index - 1]}.")
        else:
            speak("Sorry, I didn't understand your response. Please try again.")
    else:
        speak("Error fetching question. Please try again later.")

#Function to ask General knowledge 
def ask_general_knowledge_question():
    
    speak("Sure, let's test your general knowledge.")
    get_and_speak_general_knowledge_question()


# Function to say Goodbye
def say_goodbye():
    
    responses = ["Goodbye!", "See you later!", "Farewell!"]
    speak(random.choice(responses))

# Main function to handle user interactions            
def main():
    
    speak("Hi, I am your Alexa. What can I do for you?")
    time.sleep(2)
    
    while True:
        
        command = listen()
        print(f"User command : {command}")
        
            
        # Perform actions based on user's command
        
        if "hi" in command or "hello" in command:
            greet()
        
        elif "how are you" in command:
            respond_to_well_being()
    
        elif "time" in command:
            get_time()
            
        elif "play music" in command:
            play_music()
            
        elif "search" in command:
            search()
            
        elif "riddle" in command:
            riddles = get_random_riddles()
            
            for idx, riddle in enumerate(riddles, 1):
                speak(f"{riddle}")
                time.sleep(2)
                
        elif "reminder" in command:
            set_reminder()
        
        elif "send email" in command:
            send_email(to_email, subject, body)
            
        elif "weather" in command:
            speak("Sure, which city would you like the weather for?")
            city = listen()
            print(city)
            get_weather(city)
            
        elif "general knowledge" in command or "knowledge" in command:
            ask_general_knowledge_question()

        elif "goodbye" in command or "bye" in command:
            say_goodbye()
            break
            
        elif "exit" in command:
            speak("Thank you! Have a great day!")
            break
        
        else:
            speak("Sorry, I didn't understand that. Can you please repeat?")     
           
        
if __name__ == "__main__":
    
    main()
    
    
     

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
Listening....
User command : hi Alexa
Listening....
User command : how are you
Listening....
User command : what's the time now
Listening....
User command : play music
Listening....
Listening....
User command : search Alexa
Listening....
Sorry, I didn't understand that
Listening....
User command : search Alexa
Listening....
Listening....
User command : say riddles
Listening....
User command : send email
Email sent successfully!
Listening....
User command : check weather
Listening....
Coimbatore
{'coord': {'lon': 76.9667, 'lat': 11}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 25.88, 'feels_like': 26.56, 'temp_min': 25.88, 'temp_max': 28.62, 'pressure': 1013, 'humidity': 78}, 'visibility': 6000, 'wind': {'speed': 2.57, 'deg': 10}, 'clouds': {'all': 75}, 'dt': 1704706769, 'sys': {'type': 1, 'id